In [1]:
from __future__ import print_function
import numpy as np

from bqplot import (
    Axis, ColorAxis, LinearScale, DateScale, DateColorScale, OrdinalScale,
    OrdinalColorScale, ColorScale, Scatter, Lines, Figure, Tooltip
)

import ipywidgets as widgets

# Using one plot as a control for another

## Final widget will look like this

![fourier series widget](images/plot-as-control.gif)



In [2]:
N_fourier_components = 10
x_data = np.arange(N_fourier_components) + 1
y_data = np.random.uniform(low=-1, high=1, size=N_fourier_components)

In [3]:
sc_x = LinearScale()
sc_y = LinearScale(min=-1.0, max=1.0)

scat = Scatter(x=x_data, y=y_data, scales={'x': sc_x, 'y': sc_y}, colors=['orange'],
               enable_move=True)
scat.restrict_y = True

ax_x = Axis(scale=sc_x)
ax_y = Axis(scale=sc_y, tick_format='0.2f', orientation='vertical')

amplitude_control = Figure(marks=[scat], axes=[ax_x, ax_y], 
                           title='Fourier amplitudes')
amplitude_control.layout.width = '400px'
amplitude_control

A Jupyter Widget

In [4]:
print(scat.y)

[-0.04362407  0.26273339 -0.85852745  0.42058985  0.82719331 -0.34148466
  0.5125198   0.59454853 -0.47458763 -0.77869135]


In [5]:
print(y_data)

[-0.04362407  0.26273339 -0.85852745  0.42058985  0.82719331 -0.34148466
  0.5125198   0.59454853 -0.47458763 -0.77869135]


In [13]:
def fourier_series(amplitudes):
    """
    Compute the fourier sine series given a set of amplitudes. The 
    period of the fundamental of the series is 1 and the series is 
    generated for two periods.
    """
    period = 1.0
    x = np.linspace(0, 2 * period, num=1000)
    y = np.sum(a * np.sin(2 * np.pi * (n + 1) * x / period) for n, a in enumerate(amplitudes))

    return x, y

In [17]:
lin_x = LinearScale()
lin_y = LinearScale()

line = Lines(scales={'x': lin_x, 'y': lin_y}, colors=['orange'],
               enable_move=False)

ax_x = Axis(scale=lin_x)
ax_y = Axis(scale=lin_y, tick_format='0.2f', orientation='vertical')

result = Figure(marks=[line], axes=[ax_x, ax_y], 
                title='Fourier sine series',
                animation_duration=500)
result.layout.width = '400px'

# Add some test data to make view the result
initial_amplitudes = np.zeros(10)
initial_amplitudes[0] = 1.0
line.x, line.y = fourier_series(initial_amplitudes)

result

In [18]:
amplitude_control.marks[0].y = initial_amplitudes

In [19]:
box = widgets.HBox(children=[amplitude_control, result])
box

## Looks good, connect things up

In [10]:
def update_line(change):
    new_x, new_y = fourier_series(change['new'])
    line.x = new_x
    line.y = new_y
    
# React to changes in the y value....
scat.observe(update_line, names=['y'])